In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ.get('SENDGRID_ALT_KEY')

In [2]:
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))


In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page': 1, 'page_size': 1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
print(response.body)
#print(response.headers)


In [ ]:
# Sendgrid retrieve recipients
import os
from sendgrid import SendGridAPIClient


sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.marketing.contacts.exports.post()
id = response.body.decode("utf-8")
id

In [5]:
from sendgrid import SendGridAPIClient
import os
import json
import time
import pandas as pd
import requests
from io import StringIO

sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.marketing.contacts.exports._(json.loads(id)['id']).get()
time.sleep(10)
urls = response.body.decode("utf-8")
download = json.loads(urls)["urls"][0]
req = requests.get(download)
df = pd.read_csv(StringIO(req.text))
sg_emails = df['EMAIL'].tolist()

parse index.xml to get latest feed item

In [ ]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(newsletter.title)
print(newsletter.description)

In [ ]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + re.sub(r'"', r'\"', str(s.find('p'))) + '"'
    if s.find('ul'):
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('li').find_next_siblings('li')]) + ']'
    else:
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
emails = [] #..fill as needed
emails = sg_emails
for email in emails:
	sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
	message = Mail(from_email=os.environ.get('SENDGRID_SENDER_EMAIL'), to_emails=email)
	message.dynamic_template_data = eval(handlebar)
	message.template_id = os.environ.get('SENDGRID_TEMPLATE_ID')
	response = sg.send(message)
	print(response.status_code, email)

In [ ]:
sg_emails